In [ ]:
!pip install transformers
!pip install emoji
!pip install langdetect

In [ ]:
import os

import numpy as np
print('numpy', np.__version__)
import pandas as pd
print('pandas', pd.__version__)
from langdetect import detect
import random
import re

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
#from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

from transformers import AutoTokenizer, BertTokenizer, BertForSequenceClassification

from tqdm import tqdm #progress bar

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

numpy 1.21.6
pandas 1.3.5


# https://www.analyticsvidhya.com/blog/2021/07/performing-sentiment-analysis-with-naive-bayes-classifier/

In [ ]:
from sklearn.model_selection import train_test_split
import joblib
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My Drive/Twitter

/content/drive/My Drive/Twitter


In [ ]:
data = pd.read_csv('all_tweets_english.tsv', delimiter='\t', index_col=0, on_bad_lines='skip')

In [ ]:
data['label'] = 0
data.loc[data['pos'] == 1,'label'] = 1
data.loc[data['neg'] == 1,'label'] = -1

In [ ]:
def preprocess_data(data):
    # Convert text to lowercase
    data['text'] = data['text'].str.replace('/srs',' ')
    data['text'] = data['text'].str.replace('/nbh',' ')
    data['text'] = data['text'].str.replace('/pos',' ')
    data['text'] = data['text'].str.replace('/hyp',' ')
    data['text'] = data['text'].str.replace('/neg',' ')
    return data
 
data = data.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
data['text'].str.lower()
 
preprocess_data(data)
srs_df = data.loc[data['srs'] == '1']
nbh_df = data.loc[data['nbh'] == '1']
hyp_df = data.loc[data['hyp'] == '1']
pos_df = data.loc[data['pos'] == '1']
neg_df = data.loc[data['neg'] == '1']

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
x = data['text']
y = data['label']
x, x_test, y, y_test = train_test_split(x,y, stratify=y, test_size=0.25, random_state=42)

In [ ]:
vec = CountVectorizer(stop_words='english', max_features = 12000)
x = vec.fit_transform(x).toarray()
x_test = vec.transform(x_test).toarray()

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x, y)

MultinomialNB()

In [ ]:
model.score(x_test, y_test)

0.7022315946826118

In [ ]:
y_predicted = model.predict(x_test)

In [ ]:
import sklearn

sklearn.metrics.f1_score(y_test, y_predicted, average='micro')

0.7022315946826118

In [ ]:
sklearn.metrics.f1_score(y_test, y_predicted, average='weighted')

0.6895550725683741

In [ ]:
sklearn.metrics.f1_score(y_test, y_predicted, average='macro')

0.5807716646210817

In [ ]:
target_names = ['class 0', 'class 1', 'class 2']
print(sklearn.metrics.classification_report(y_test, y_predicted, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.41      0.22      0.29      3895
     class 1       0.69      0.65      0.67     13899
     class 2       0.74      0.83      0.78     19668

    accuracy                           0.70     37462
   macro avg       0.61      0.57      0.58     37462
weighted avg       0.69      0.70      0.69     37462



In [ ]:
from collections import Counter
srs_count = dict(Counter(" ".join(srs_df["text"]).split()).most_common(100))
nbh_count = dict(Counter(" ".join(nbh_df["text"]).split()).most_common(100))
hyp_count = dict(Counter(" ".join(hyp_df["text"]).split()).most_common(100))
pos_count = dict(Counter(" ".join(pos_df["text"]).split()).most_common(100))
neg_count = dict(Counter(" ".join(neg_df["text"]).split()).most_common(100))

In [ ]:
srs_set = set(srs_count.keys())
nbh_set = set(nbh_count.keys())
hyp_set = set(hyp_count.keys())
pos_set = set(pos_count.keys())
neg_set = set(neg_count.keys())

In [ ]:
srs_res = srs_set.difference(nbh_set, hyp_set, pos_set, neg_set)
nbh_res = nbh_set.difference(srs_set, hyp_set, pos_set, neg_set)
hyp_res = hyp_set.difference(nbh_set, srs_set, pos_set, neg_set)
pos_res = pos_set.difference(nbh_set, hyp_set, srs_set, neg_set)
neg_res = neg_set.difference(nbh_set, hyp_set, pos_set, srs_set)

In [ ]:
srs_li = list(srs_count.keys())
nbh_li = list(nbh_count.keys())
hyp_li = list(hyp_count.keys())
pos_li = list(pos_count.keys())
neg_li = list(neg_count.keys())
 
for k in srs_li:
  if k not in srs_res:
    del srs_count[k]
 
for k in nbh_li:
  if k not in nbh_res:
    del nbh_count[k]
 
for k in hyp_li:
  if k not in hyp_res:
    del hyp_count[k]
 
for k in pos_li:
  if k not in pos_res:
    del pos_count[k]
 
for k in neg_li:
  if k not in neg_res:
    del neg_count[k]

In [ ]:
srs_count

{'?': 1375,
 'actually': 1459,
 'any': 1079,
 'anything': 1065,
 'drop': 1728,
 'guys': 1296,
 'pls': 1897,
 'tweet': 1195}

In [ ]:
nbh_count

{'bc': 128,
 'off': 118,
 'person': 128,
 'ppl': 220,
 'stop': 222,
 'talk': 123,
 'then': 143,
 'yall': 204,
 "you're": 126,
 'youre': 124}

In [ ]:
hyp_count

{'#kernel': 6,
 '/hj': 8,
 '/lh': 4,
 '/rh': 5,
 '/s': 5,
 '=': 60,
 'The': 7,
 'again': 4,
 'arch/arm64/kvm': 6,
 'connotation': 6,
 'er': 15,
 'ever': 11,
 'every': 5,
 'has': 7,
 'here': 5,
 'hyperbole': 5,
 'joke': 7,
 'myself': 9,
 'question': 7,
 'says': 4,
 'something': 6,
 'than': 7,
 'warning:': 5}

In [ ]:
pos_count

{'&lt;3': 1400,
 ':(': 2524,
 ':((': 1535,
 'AND': 1681,
 'IM': 1556,
 'IS': 1670,
 'MY': 1559,
 'SO': 2696,
 'THE': 2359,
 'THIS': 1641,
 'YOU': 1517,
 'art': 1769,
 'cry': 3547,
 'crying': 2620,
 'good': 1575,
 'him': 1810,
 'his': 1582,
 'look': 1584,
 'oh': 1637,
 'omg': 1476,
 'very': 1601}

In [ ]:
neg_count

{'//': 361, 'also': 318, 'had': 286}